# SENSITIVITY_MATRIX

## Overview
The `SENSITIVITY_MATRIX` function computes how the outputs of a model change with respect to changes in parameters or initial conditions. It provides a sensitivity matrix or vector, enabling business users to assess the impact of parameter variations on model results directly in Excel. This function leverages the [CasADi](https://github.com/casadi/casadi) package for symbolic and algorithmic differentiation. For more information, see the [CasADi documentation](https://web.casadi.org/).

From a technical perspective, the function uses symbolic differentiation to compute the Jacobian matrix of the model output with respect to the specified parameters. Given a model $f(\mathbf{x}, \mathbf{a})$ where $\mathbf{x}$ are variables and $\mathbf{a}$ are parameters, the sensitivity matrix $S$ is defined as:

```math
S = \frac{\partial f}{\partial \mathbf{a}}
```

This means each entry $S_{ij}$ represents the partial derivative of the $i$-th output with respect to the $j$-th parameter, evaluated at the provided values. The CasADi library constructs the symbolic computation graph and efficiently evaluates these derivatives using algorithmic differentiation, ensuring both accuracy and performance for complex models.

This example function is provided as-is without any representation of accuracy.

## Usage
To use the function in Excel:
```excel
=SENSITIVITY_MATRIX(model, variables, parameters, [variable_names], [parameter_names])
```
- `model` (string, required): Symbolic expression for the model output as a function of variables and parameters.
- `variables` (2D list of float, required): Values for the model's variables at which to compute sensitivities.
- `parameters` (2D list of float, required): Values for the model's parameters at which to compute sensitivities.
- `variable_names` (2D list of string, optional): Names of the variables (if not inferred from the model).
- `parameter_names` (2D list of string, optional): Names of the parameters (if not inferred from the model).

The function returns a sensitivity matrix or vector (list[list[float]]) showing how outputs change with respect to parameters, or a string error message if the calculation fails or input is invalid.

## Examples

**Example 1: Financial Forecasting**

A financial analyst wants to understand how changes in interest rate and growth rate parameters affect projected revenue.

In Excel:
```excel
=SENSITIVITY_MATRIX("x*a + y*b", {100, 200}, {0.05, 0.03}, {"x", "y"}, {"a", "b"})
```
Expected output:

|      | a    | b    |
|------|------|------|
|      | 100  | 200  |

**Example 2: Engineering Process Control**

An engineer analyzes how variations in process parameters affect system output.

In Excel:
```excel
=SENSITIVITY_MATRIX("x**2 + a*y", {2, 3}, {1.5}, {"x", "y"}, {"a"})
```
Expected output:

|      | a    |
|------|------|
|      | 3.0  |

In [ ]:
import casadi as ca

def sensitivity_matrix(model, variables, parameters, variable_names=None, parameter_names=None):
    """
    Computes the sensitivity matrix of a model output with respect to parameters using CasADi.

    Args:
        model (str): Symbolic expression for the model output (e.g., "x**2 + a*y").
        variables (list[list[float]]): Values for the model's variables (e.g., [[1.0, 2.0]]).
        parameters (list[list[float]]): Values for the model's parameters (e.g., [[0.5, 1.5]]).
        variable_names (list[list[str]], optional): Names of the variables (e.g., [["x", "y"]]).
        parameter_names (list[list[str]], optional): Names of the parameters (e.g., [["a"]]).

    Returns:
        list[list[float]]: Sensitivity matrix or vector, or error message string.

    This example function is provided as-is without any representation of accuracy.
    """
    try:
        if not isinstance(model, str):
            return "model must be a string."
        if not (isinstance(variables, list) and len(variables) > 0 and isinstance(variables[0], list)):
            return "variables must be a 2D list of floats."
        if not (isinstance(parameters, list) and len(parameters) > 0 and isinstance(parameters[0], list)):
            return "parameters must be a 2D list of floats."
        var_vals = variables[0]
        param_vals = parameters[0]
        if variable_names is not None:
            if not (isinstance(variable_names, list) and len(variable_names) > 0 and isinstance(variable_names[0], list)):
                return "variable_names must be a 2D list of strings."
            var_names = variable_names[0]
        else:
            var_names = [f"x{i+1}" for i in range(len(var_vals))]
        if parameter_names is not None:
            if not (isinstance(parameter_names, list) and len(parameter_names) > 0 and isinstance(parameter_names[0], list)):
                return "parameter_names must be a 2D list of strings."
            param_names = parameter_names[0]
        else:
            param_names = [f"a{i+1}" for i in range(len(param_vals))]
        if len(var_names) != len(var_vals):
            return "Number of variable names and values must match."
        if len(param_names) != len(param_vals):
            return "Number of parameter names and values must match."
        sym_vars = [ca.MX.sym(name) for name in var_names]
        sym_params = [ca.MX.sym(name) for name in param_names]
        vars_dict = {name: sym_vars[i] for i, name in enumerate(var_names)}
        params_dict = {name: sym_params[i] for i, name in enumerate(param_names)}
        try:
            expr = eval(model, {**vars_dict, **params_dict, 'ca': ca})
        except Exception as e:
            return f"Invalid model expression: {str(e)}"
        S = ca.jacobian(expr, ca.vertcat(*sym_params))
        sens_func = ca.Function('sens_func', sym_vars + sym_params, [S])
        result_matrix = sens_func(*(var_vals + param_vals))
        if isinstance(result_matrix, ca.DM):
            return result_matrix.full().tolist()
        else:
            return "Error during CasADi calculation: Unexpected result type."
    except ca.CasadiException as e:
        return f"Error during CasADi calculation: {e}"
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
import pytest

demo_cases = [
    ["x*a + y*b", [[100, 200]], [[0.05, 0.03]], [["x", "y"]], [["a", "b"]], [[100.0, 200.0]]],
    ["x**2 + a*y", [[2, 3]], [[1.5]], [["x", "y"]], [["a"]], [[3.0]]],
]

def is_valid_type(val):
    if isinstance(val, str):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, float) or isinstance(x, int) for x in row) for row in val)
    return False

@pytest.mark.parametrize("model, variables, parameters, variable_names, parameter_names, expected", demo_cases)
def test_demo_cases(model, variables, parameters, variable_names, parameter_names, expected):
    result = sensitivity_matrix(model, variables, parameters, variable_names, parameter_names)
    print(f"test_demo_cases output for {model}: {result}")
    assert is_valid_type(result)

def test_invalid_model():
    result = sensitivity_matrix("x**2 +", [[2, 3]], [[1.5]], [["x", "y"]], [["a"]])
    assert isinstance(result, str) and ("error" in result.lower() or "invalid" in result.lower())

ipytest.run('-s')

In [ ]:
import gradio as gr

def gradio_sensitivity(model, variables, parameters, variable_names, parameter_names):
    result = sensitivity_matrix(model, variables, parameters, variable_names, parameter_names)
    if isinstance(result, str):
        return [[result]]
    return result

demo = gr.Interface(
    fn=gradio_sensitivity,
    inputs=[
        gr.Textbox(label="Model Expression", value=demo_cases[0][0]),
        gr.DataFrame(label="Variables", type="array", value=demo_cases[0][1], headers=demo_cases[0][3][0]),
        gr.DataFrame(label="Parameters", type="array", value=demo_cases[0][2], headers=demo_cases[0][4][0]),
        gr.DataFrame(label="Variable Names", type="array", value=demo_cases[0][3], headers=["Variable Names"], visible=False),
        gr.DataFrame(label="Parameter Names", type="array", value=demo_cases[0][4], headers=["Parameter Names"], visible=False),
    ],
    outputs=gr.DataFrame(label="Sensitivity Matrix or Error Message", type="array", headers=["a", "b"]),
    examples=demo_cases,
    flagging_mode="never",
    fill_width=True,
)
demo.launch()